Autor : gregory Wallace (gregorwa@gmail.com) // env : python 3.6 

For the first part , we needed to perform a supervised binary classification with images as input.

Kaggle was used with a default python3 kernel. The score achieved was 94,3%.

You can somehow compare Kaggle to google colab. But it is also a good place to find some Machine Learning codes with algorithms suitable to your topic thanks to a nice & open community. 

Why Kaggle ? Because last year, there was the WIDS (Women in Data Science) competition with a very similar use case : to see if a picture has oil palm or not. So a notebook had already been created.  

For the current challenge, because of the size constraints, only a small sample of the pictures were loaded into kaggle. (900 train pics, 300 test pics) . The full holdout (3073 pics) was of course loaded.

The libraty used was FastAI (CNN & Resnet API) https://www.fast.ai/. 
FastAI have wrapper objects that make the exercise more affordable to people with little background in deep learning. It’s a ‘top down’ approach to get into deep learning.

However it is not so easy once you dig deeper into the objects of the library. They have good free tutorials. But you will need to invest the time nevertheless. 

You will have to set GPU to 'on' before executing the script. 

In [ ]:
import os

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import fastai
from fastai.vision import *

work_dir = Path('/kaggle/working/')
path = Path('../input')
#fichiers où sont les photos : 
train = path /'parkings/challenge_light/TRAINLIGHT/'
test =  path/'parkings/challenge_light/TESTLIGHT/'
holdout = path/'parkings/challenge_light/HOLDOUT/'

#on entraine et on teste sur un petit échatilllon puis on prédira sur le holdout complet
sample_sub = path/'parkings/challenge_light/holdout.csv'
train_labels = path/'parkings/challenge_light/trainlight.csv'  
df = pd.read_csv('/kaggle/input/parkings/challenge_light/trainlight.csv', sep = ';')
(df.has_parking==1).sum()

In [ ]:
df_sample = pd.read_csv(sample_sub)

In [ ]:
test_names = [f for f in test.iterdir()]
holdout_names = [f for f in holdout.iterdir()]

In [ ]:
#cf la doc fastAI pour les ImageList
src = (ImageList.from_df(df, path, folder=train)
      .split_by_rand_pct(0.2, seed=2019)
      .label_from_df('has_parking')
      .add_test(test_names+holdout_names))

In [ ]:
#"To get a set of transforms with default values that work pretty well in a wide range of tasks, 
#it's often easiest to use get_transforms."
data =  (src.transform(get_transforms(), size=128)
         .databunch(bs=64)
         .normalize(imagenet_stats))

In [ ]:
#check that labels are correctly linked to images
data.show_batch(4, figsize=(10,7))

In [ ]:
learn = cnn_learner(data, models.resnet18, 
                   metrics=[accuracy], model_dir='/kaggle/working/models')

In [ ]:
learn.lr_find(); learn.recorder.plot()

In [ ]:
lr = 1e-1

In [ ]:
learn.fit_one_cycle(6, lr)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-3))

In [ ]:
learn.save('128')

In [ ]:
#This was working perfectly some minutes ago!
from sklearn.metrics import roc_auc_score
def auc_score(preds,targets):
    return torch.tensor(roc_auc_score(targets,preds[:,1]))

In [ ]:
p,t = learn.get_preds()
auc_score(p,t)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

In [ ]:
interp.plot_top_losses(16, figsize=(15,11))

In [ ]:
p,t = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
p = to_np(p); p.shape

In [ ]:
p[:,1]

In [ ]:
ids = np.array([f.name for f in (test_names+holdout_names)]);ids.shape

In [ ]:
np.stack([ids, p[:,1]], axis=1)

In [ ]:
df_sample.columns

In [ ]:
#We only recover the probs of having parking (column 1)
sub = pd.DataFrame(np.stack([ids, p[:,1]], axis=1), columns=['id','has_parking' ])

In [ ]:
sub.sample(13)

In [ ]:
#check shape
sub.info()

In [ ]:
#convert probabilities to 1-0
sub['has_parking'] = sub['has_parking'].astype(float)
sub['has_parking'] = sub['has_parking'].round(0).astype(int)
sub['has_parking']

In [ ]:
#we had to convert to jpg because fastai was not happy with tif
sub['id'] = sub['id'].str.replace('jpg', 'tif')

In [ ]:
sub.sample(12)

In [ ]:
#ready to go ! 
sub.to_csv(work_dir/'sub.csv', index=False, sep = ';')